# Caching Tools Usage Notebook

An example of heavy computation

In [1]:
def fibonacci(n):
    if n <= 1:
        return n
    else:
        return fibonacci(n-1) + fibonacci(n-2)

%time fibonacci(37)

CPU times: user 3.38 s, sys: 0 ns, total: 3.38 s
Wall time: 3.38 s


24157817

## CacheToDisk Usage

Create an instance of the `CacheToDisk` class with desired parameters

In [2]:
from pathlib import Path
from cachingtools import CacheToDisk


cache_directory = Path('cache') # parent directory for the cache files
compress = True # compress the cache files
capacity = 1_000_000 # maximum number of bytes to store in the cache (1 MB)
cache_to_disk = CacheToDisk(cache_directory, compress=compress, capacity=capacity)
cache_to_disk

CacheToDisk(directory=cache, compress=True, capacity=1000000)

Use the instance as a decorator for the function you want to cache

In [3]:
# The following function will be cached to disk in the "cache/fibonacci" directory
@cache_to_disk("fibonacci")
def fibonacci_cached(n):
    if n <= 1:
        return n
    else:
        return fibonacci_cached(n-1) + fibonacci_cached(n-2)

%time fibonacci_cached(37)

CPU times: user 24.1 ms, sys: 12 ms, total: 36.2 ms
Wall time: 35.9 ms


24157817

If you want to cache a method of a class do as follows:

In [4]:
class ExampleClass:
    def compute(self, x):
        return x**2

# create a subclass of ExampleClass that caches the results of the compute method

class ExampleClassCached(ExampleClass):
    # cache the to disk in the "cache/ExampleClass.compute" directory
    @cache_to_disk("ExampleClass.compute")
    def compute(self, x):
        return super().compute(x)

example = ExampleClassCached()
example.compute(2)

4

You can shrink the cache directory to the desired size by calling the `shrink` method.
This method will delete least recently accessed cache files until the cache directory is
below the desired size.

In [5]:
cache_to_disk.shrink(10000) # shrink the cache to 10 KB

You can delete the cache directory by calling the `purge` method of the instance

In [6]:
cache_to_disk.purge()